In [1]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam


Using TensorFlow backend.


In [2]:
base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(10,activation='softmax')(x) #final layer with softmax activation


/usr/local/lib/python3.5/dist-packages/keras_applications/mobilenet.py:207: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


In [3]:
model=Model(inputs=base_model.input,outputs=preds)

In [4]:
for i,layer in enumerate(model.layers):
  print(i,layer.name)

0 input_1
1 conv1_pad
2 conv1
3 conv1_bn
4 conv1_relu
5 conv_dw_1
6 conv_dw_1_bn
7 conv_dw_1_relu
8 conv_pw_1
9 conv_pw_1_bn
10 conv_pw_1_relu
11 conv_pad_2
12 conv_dw_2
13 conv_dw_2_bn
14 conv_dw_2_relu
15 conv_pw_2
16 conv_pw_2_bn
17 conv_pw_2_relu
18 conv_dw_3
19 conv_dw_3_bn
20 conv_dw_3_relu
21 conv_pw_3
22 conv_pw_3_bn
23 conv_pw_3_relu
24 conv_pad_4
25 conv_dw_4
26 conv_dw_4_bn
27 conv_dw_4_relu
28 conv_pw_4
29 conv_pw_4_bn
30 conv_pw_4_relu
31 conv_dw_5
32 conv_dw_5_bn
33 conv_dw_5_relu
34 conv_pw_5
35 conv_pw_5_bn
36 conv_pw_5_relu
37 conv_pad_6
38 conv_dw_6
39 conv_dw_6_bn
40 conv_dw_6_relu
41 conv_pw_6
42 conv_pw_6_bn
43 conv_pw_6_relu
44 conv_dw_7
45 conv_dw_7_bn
46 conv_dw_7_relu
47 conv_pw_7
48 conv_pw_7_bn
49 conv_pw_7_relu
50 conv_dw_8
51 conv_dw_8_bn
52 conv_dw_8_relu
53 conv_pw_8
54 conv_pw_8_bn
55 conv_pw_8_relu
56 conv_dw_9
57 conv_dw_9_bn
58 conv_dw_9_relu
59 conv_pw_9
60 conv_pw_9_bn
61 conv_pw_9_relu
62 conv_dw_10
63 conv_dw_10_bn
64 conv_dw_10_relu
65 conv_pw_10

In [5]:
#for layer in model.layers:
#    layer.trainable=False
# or if we want to set the first 20 layers of the network to be non-trainable
for layer in model.layers[:90]:
    layer.trainable=False
for layer in model.layers[90:]:
    layer.trainable=True

In [6]:
#train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

#train_generator=train_datagen.flow_from_directory('Tobacco/train/',
#                                                 target_size=(224,224),
#                                                 color_mode='rgb',
#                                                 batch_size=32,
#                                                 class_mode='categorical',
#                                                 shuffle=True)

In [7]:
#train_datagen = ImageDataGenerator(rescale=1./255,
#    shear_range=0.2,
#    zoom_range=0.2,
#    horizontal_flip=True,
#    validation_split=0.2) # set validation split

train_datagen=ImageDataGenerator(validation_split=0.2, 
    preprocessing_function=preprocess_input) #included in our dependencies

train_data_dir = 'Tobacco/train/'
img_height, img_width = 224,224
color_mode = 'rgb'
batch_size = 32

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    color_mode=color_mode,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(img_height, img_width),
    color_mode=color_mode,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data


Found 2230 images belonging to 10 classes.
Found 553 images belonging to 10 classes.


In [8]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

#step_size_train=train_generator.n//train_generator.batch_size
#model.fit_generator(generator=train_generator,
#                   steps_per_epoch=step_size_train,
#                   epochs=10, validation_split=0.2)

nb_epochs = 100

model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = nb_epochs)


Epoch 1/100
69/69 [==============================] - 36s 515ms/step - loss: 1.5251 - acc: 0.4882 - val_loss: 2.1751 - val_acc: 0.2243
Epoch 2/100
69/69 [==============================] - 28s 413ms/step - loss: 1.0600 - acc: 0.6369 - val_loss: 2.3598 - val_acc: 0.1785
Epoch 3/100
69/69 [==============================] - 29s 417ms/step - loss: 0.9032 - acc: 0.6973 - val_loss: 2.5445 - val_acc: 0.1862
Epoch 4/100
69/69 [==============================] - 26s 373ms/step - loss: 0.8153 - acc: 0.7250 - val_loss: 2.7762 - val_acc: 0.0940
Epoch 5/100
69/69 [==============================] - 28s 403ms/step - loss: 0.7612 - acc: 0.7397 - val_loss: 3.0444 - val_acc: 0.1209
Epoch 6/100
69/69 [==============================] - 29s 421ms/step - loss: 0.6838 - acc: 0.7666 - val_loss: 2.7844 - val_acc: 0.1708
Epoch 7/100
69/69 [==============================] - 25s 366ms/step - loss: 0.6664 - acc: 0.7680 - val_loss: 2.8868 - val_acc: 0.1843
Epoch 8/100
69/69 [==============================] - 27s 388ms

Epoch 62/100
69/69 [==============================] - 27s 392ms/step - loss: 0.0506 - acc: 0.9887 - val_loss: 7.4401 - val_acc: 0.1152
Epoch 63/100
69/69 [==============================] - 26s 373ms/step - loss: 0.0468 - acc: 0.9887 - val_loss: 6.8823 - val_acc: 0.1286
Epoch 64/100
69/69 [==============================] - 30s 433ms/step - loss: 0.0480 - acc: 0.9856 - val_loss: 7.2791 - val_acc: 0.1612
Epoch 65/100
69/69 [==============================] - 28s 411ms/step - loss: 0.0736 - acc: 0.9799 - val_loss: 6.8250 - val_acc: 0.2111
Epoch 66/100
69/69 [==============================] - 27s 396ms/step - loss: 0.0836 - acc: 0.9742 - val_loss: 6.3659 - val_acc: 0.1708
Epoch 67/100
69/69 [==============================] - 25s 369ms/step - loss: 0.0372 - acc: 0.9887 - val_loss: 6.7752 - val_acc: 0.1766
Epoch 68/100
69/69 [==============================] - 25s 366ms/step - loss: 0.0832 - acc: 0.9742 - val_loss: 6.3647 - val_acc: 0.1612
Epoch 69/100
69/69 [==============================] - 2

KeyboardInterrupt: 